In [1]:
# David Ouyang 12/5/2019

# Notebook which:
# 1. Downloads weights
# 2. Initializes model and imports weights
# 3. Performs test time evaluation of videos (already preprocessed with ConvertDICOMToAVI.ipynb)

import re
import os, os.path
from os.path import splitext
import pydicom as dicom
import numpy as np
from pydicom.uid import UID, generate_uid
import shutil
from multiprocessing import dummy as multiprocessing
import time
import subprocess
import datetime
from datetime import date
import sys
import cv2
import matplotlib.pyplot as plt
import sys
from shutil import copy
import math
import torch
import torchvision
import echonet
import wget 

destinationFolder = "C:\\Users\\Windows\\Dropbox\\Echo Research\\CodeBase\\Output"
videosFolder = "C:\\Users\\Windows\\Dropbox\\Echo Research\\CodeBase\\a4c-video-dir"
DestinationForWeights = "C:\\Users\\Windows\\Dropbox\\Echo Research\\CodeBase\\EchoNetDynamic-Weights"

In [2]:
# Download model weights

if os.path.exists(DestinationForWeights):
    print("The weights are at", DestinationForWeights)
else:
    print("Creating folder at ", DestinationForWeights, " to store weights")
    os.mkdir(DestinationForWeights)
    
segmentationWeightsURL = 'https://github.com/douyang/EchoNetDynamic/releases/download/v1.0.0/deeplabv3_resnet50_random.pt'
ejectionFractionWeightsURL = 'https://github.com/douyang/EchoNetDynamic/releases/download/v1.0.0/r2plus1d_18_32_2_pretrained.pt'


if not os.path.exists(os.path.join(DestinationForWeights, os.path.basename(segmentationWeightsURL))):
    print("Downloading Segmentation Weights, ", segmentationWeightsURL," to ",os.path.join(DestinationForWeights,os.path.basename(segmentationWeightsURL)))
    filename = wget.download(segmentationWeightsURL, out = DestinationForWeights)
else:
    print("Segmentation Weights already present")
    
if not os.path.exists(os.path.join(DestinationForWeights, os.path.basename(ejectionFractionWeightsURL))):
    print("Downloading EF Weights, ", ejectionFractionWeightsURL," to ",os.path.join(DestinationForWeights,os.path.basename(ejectionFractionWeightsURL)))
    filename = wget.download(ejectionFractionWeightsURL, out = DestinationForWeights)
else:
    print("EF Weights already present")
        


The weights are at C:\Users\Windows\Dropbox\Echo Research\CodeBase\EchoNetDynamic-Weights
Segmentation Weights already present
EF Weights already present


In [3]:
# Initialize and Run EF model

frames = 32
period = 2
batch_size = 20
model = torchvision.models.video.r2plus1d_18(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, 1)


device = torch.device("cuda")
if device.type == "cuda":
    model = torch.nn.DataParallel(model)
model.to(device)

# try some random weights: final_r2+1d_model_regression_EF_sgd_skip1_32frames.pth.tar
# scp ouyangd@arthur2:~/Echo-Tracing-Analysis/final_r2+1d_model_regression_EF_sgd_skip1_32frames.pth.tar "C:\Users\Windows\Dropbox\Echo Research\CodeBase\EchoNetDynamic-Weights"
#Weights = "final_r2+1d_model_regression_EF_sgd_skip1_32frames.pth.tar"

print("loading weights from ", os.path.join(DestinationForWeights, "r2plus1d_18_32_2_pretrained"))

checkpoint = torch.load(os.path.join(DestinationForWeights, os.path.basename(ejectionFractionWeightsURL)))
model.load_state_dict(checkpoint['state_dict'])

output = os.path.join(destinationFolder, "ef_output.csv")

ds = echonet.datasets.Echo(split = "external_test", external_test_location = videosFolder, crops="all")
print(ds.split, ds.fnames)

mean, std = echonet.utils.get_mean_and_std(ds)

kwargs = {"target_type": "EF",
          "mean": mean,
          "std": std,
          "length": frames,
          "period": period,
          }

ds = echonet.datasets.Echo(split = "external_test", external_test_location = videosFolder, **kwargs, crops="all")

test_dataloader = torch.utils.data.DataLoader(ds, batch_size = 1, num_workers = 5, shuffle = True, pin_memory=(device.type == "cuda"))
loss, yhat, y = echonet.utils.video.run_epoch(model, test_dataloader, "test", None, device, save_all=True, blocks=25)

with open(output, "w") as g:
    for (filename, pred) in zip(ds.fnames, yhat):
        for (i,p) in enumerate(pred):
            g.write("{},{},{:.4f}\n".format(filename, i, p))


loading weights from  C:\Users\Windows\Dropbox\Echo Research\CodeBase\EchoNetDynamic-Weights\r2plus1d_18_32_2_pretrained
external_test C:\Users\Windows\Dropbox\Echo Research\CodeBase\a4c-video-dir
['0X1A05DFFFCAFB253B.avi', '0X1A0A263B22CCD966.avi', '0X1A2A76BDB5B98BED.avi', '0X1A2C60147AF9FDAE.avi', '0X1A2E9496910EFF5B.avi', '0X1A3D565B371DC573.avi', '0X1A3E7BF1DFB132FB.avi', '0X1A5FAE3F9D37794E.avi', '0X1A6ACFE7B286DAFC.avi', '0X1A8D85542DBE8204.avi']
external_test ['0X1A05DFFFCAFB253B.avi', '0X1A0A263B22CCD966.avi', '0X1A2A76BDB5B98BED.avi', '0X1A2C60147AF9FDAE.avi', '0X1A2E9496910EFF5B.avi', '0X1A3D565B371DC573.avi', '0X1A3E7BF1DFB132FB.avi', '0X1A5FAE3F9D37794E.avi', '0X1A6ACFE7B286DAFC.avi', '0X1A8D85542DBE8204.avi']


100%|██████████| 10/10 [00:04<00:00,  2.06it/s]


external_test C:\Users\Windows\Dropbox\Echo Research\CodeBase\a4c-video-dir
['0X1A05DFFFCAFB253B.avi', '0X1A0A263B22CCD966.avi', '0X1A2A76BDB5B98BED.avi', '0X1A2C60147AF9FDAE.avi', '0X1A2E9496910EFF5B.avi', '0X1A3D565B371DC573.avi', '0X1A3E7BF1DFB132FB.avi', '0X1A5FAE3F9D37794E.avi', '0X1A6ACFE7B286DAFC.avi', '0X1A8D85542DBE8204.avi']


100%|██████████| 10/10 [00:28<00:00,  2.88s/it, 3524.12 (3874.05) / 0.00]


In [9]:
ejectionFractionWeightsURL
ejectionFractionWeightsURL

'https://github.com/douyang/EchoNetDynamic/releases/download/v1.0.0/r2plus1d_18_32_2_pretrained.pt'

In [36]:
kwargs

NameError: name 'kwargs' is not defined

In [16]:
print(checkpoint)

{'epoch': 43,
 'state_dict': OrderedDict([('module.stem.0.weight',
               tensor([[[[[ 4.4223e-03,  1.0617e-02,  2.0713e-02,  ...,  1.9927e-02,
                            1.6080e-03,  2.2324e-02],
                          [-2.8795e-02, -1.7580e-02,  9.4286e-03,  ...,  5.7126e-02,
                            4.0897e-02,  4.8279e-02],
                          [-5.0920e-03, -6.4447e-03,  3.6270e-02,  ...,  7.2963e-02,
                            6.3844e-02,  8.4026e-02],
                          ...,
                          [ 2.1691e-03,  2.5524e-02,  1.0068e-01,  ...,  1.3732e-01,
                            1.2547e-01,  1.1706e-01],
                          [ 1.5962e-03,  1.5016e-02,  6.5355e-02,  ...,  1.0109e-01,
                            9.7474e-02,  6.2749e-02],
                          [-1.2907e-02, -1.4594e-02,  2.1892e-02,  ...,  5.4219e-02,
                            4.2105e-02,  3.3312e-02]]],
               
               
                        [[[ 4.6635

In [2]:
def mask(output):
    dimension = output.shape[0]
    
    # Mask pixels outside of scanning sector
    m1, m2 = np.meshgrid(np.arange(dimension), np.arange(dimension))
    

    mask = ((m1+m2)>int(dimension/2) + int(dimension/10)) 
    mask *=  ((m1-m2)<int(dimension/2) + int(dimension/10))
    mask = np.reshape(mask, (dimension, dimension)).astype(np.int8)
    maskedImage = cv2.bitwise_and(output, output, mask = mask)
    
    #print(maskedImage.shape)
    
    return maskedImage



In [3]:
def makeVideo(fileToProcess, destinationFolder):
    try:
        fileName = fileToProcess.split('\\')[-1] #\\ if windows, / if on mac or sherlock
                                                 #hex(abs(hash(fileToProcess.split('/')[-1]))).upper()

        if not os.path.isdir(os.path.join(destinationFolder,fileName)):

            dataset = dicom.dcmread(fileToProcess, force=True)
            testarray = dataset.pixel_array

            frame0 = testarray[0]
            mean = np.mean(frame0, axis=1)
            mean = np.mean(mean, axis=1)
            yCrop = np.where(mean<1)[0][0]
            testarray = testarray[:, yCrop:, :, :]

            bias = int(np.abs(testarray.shape[2] - testarray.shape[1])/2)
            if bias>0:
                if testarray.shape[1] < testarray.shape[2]:
                    testarray = testarray[:, :, bias:-bias, :]
                else:
                    testarray = testarray[:, bias:-bias, :, :]


            print(testarray.shape)
            frames,height,width,channels = testarray.shape

            fps = 30

            try:
                fps = dataset[(0x18, 0x40)].value
            except:
                print("couldn't find frame rate, default to 30")

            fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
            video_filename = os.path.join(destinationFolder, fileName + '.avi')
            out = cv2.VideoWriter(video_filename, fourcc, fps, cropSize)


            for i in range(frames):

                outputA = testarray[i,:,:,0]
                smallOutput = outputA[int(height/10):(height - int(height/10)), int(height/10):(height - int(height/10))]

                # Resize image
                output = cv2.resize(smallOutput, cropSize, interpolation = cv2.INTER_CUBIC)

                finaloutput = mask(output)


                finaloutput = cv2.merge([finaloutput,finaloutput,finaloutput])
                out.write(finaloutput)

            out.release()

        else:
            print(fileName,"hasAlreadyBeenProcessed")
    except:
        print("something filed, not sure what, have to debug", fileName)
    return 0

In [ ]:
AllA4cNames = "Input Folder Name"

count = 0
    
cropSize = (112,112)
subfolders = os.listdir(AllA4cNames)


for folder in subfolders:
    print(folder)

    for content in os.listdir(os.path.join(AllA4cNames, folder)):
        for subcontent in os.listdir(os.path.join(AllA4cNames, folder, content)):
            count += 1
            

            VideoPath = os.path.join(AllA4cNames, folder, content, subcontent)

            print(count, folder, content, subcontent)

            if not os.path.exists(os.path.join(destinationFolder,subcontent + ".avi")):
                makeVideo(VideoPath, destinationFolder)
            else:
                print("Already did this file", VideoPath)


print(len(AllA4cFilenames))